## Evaluation of SegmenterHAIS

In [1]:
import os 
import sys 
import numpy as np 
import open3d as o3d 
import matplotlib.pyplot as plt 
import plotly
import pickle 
%cd /home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
/home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts


### ScanNet

In [2]:
# 0. variable definition 
scannet_scans_dir = "/media/junting/SSD_data/ScanNet/scans"
scannet_annot_dir = "/home/junting/project_cvl/HAIS/dataset/scannetv2"
scannet_scene_name = "scene0000_00"
scannet_ply = os.path.join(scannet_scans_dir, scannet_scene_name, f"{scannet_scene_name}_vh_clean_2.ply")

In [3]:
# 1. initialize model 
from models.segmenter_hais import SegmenterHAIS
from config.config_hais import ConfigHAIS

config = ConfigHAIS()
seg_hais = SegmenterHAIS(config)

[2022-02-12 09:32:54,012  INFO  log.py  line 39  1010375]  ************************ Start Logging ************************
INFO - 2022-02-12 09:32:54,012 - log - ************************ Start Logging ************************


cuda available: True


[2022-02-12 09:32:55,638  INFO  utils.py  line 67  1010375]  Restore from /home/junting/project_cvl/HAIS/pretrain/hais_ckpt.pth
INFO - 2022-02-12 09:32:55,638 - utils - Restore from /home/junting/project_cvl/HAIS/pretrain/hais_ckpt.pth


In [4]:
# 2. load scannet data and run inference
scannet_o3d = o3d.io.read_point_cloud(scannet_ply)
down_sampled_o3d_pcl, pred_result = seg_hais.predict(scannet_o3d)

In [5]:
print(len(scannet_o3d.points))

81369


In [31]:
# 3. convert segmentation to votenet batch_pred_map_cls 
from utils.box_util import get_3d_box

pred_map_cls = []
points = np.asarray(down_sampled_o3d_pcl.points)

for inst_id, inst_label in enumerate(pred_result['label_id']):
    inst_points = points[pred_result['inst_mask'][inst_id]==1, :]
    inst_min = inst_points.min(0)
    inst_max = inst_points.max(0)
    center = (inst_min + inst_max) / 2.0
    size = inst_max - inst_min
    aabb = get_3d_box(size, 0, center)
    
    # (pred_cls, pred_box_params, score)
    pred_map_cls.append((
        pred_result['label_id'][inst_id],
        aabb,
        pred_result['inst_conf'][inst_id]
    ))

batch_pred_map_cls = [pred_map_cls]

In [7]:
# 4. load scannet gt data 
import glob 
import torch 

annot_file = os.path.join(scannet_annot_dir, 'train', f"{scannet_scene_name}_inst_nostuff.pth")
# NOTE: the point clouds has been rotated to axis-aligned frame in HAIS data preparation 
xyz_origin, rgb, label, instance_label = torch.load(annot_file) 

In [14]:
# scannet gt related definitions 
def getInstanceInfo(xyz, sem_label, instance_label):
    '''
    :param xyz: (n, 3)
    :param label: (n), int, [0~19], -100 for unlabeled data   
    :param instance_label: (n), int, (0~nInst-1, -100)
    :return: instance_num, dict
    '''
    instance_num = int(instance_label.max()) + 1
    # (n, 9), float, (cx, cy, cz, minx, miny, minz, maxx, maxy, maxz)
    instance_info = np.ones((instance_num, 9), dtype=np.float32) * -100.0  
    instance_pointnum = []   # (nInst), int
    instance_sem_label = np.zeros(instance_num, dtype=int)
    
    for i_ in range(instance_num):
        
        inst_idx_i = np.where(instance_label == i_)

        # instance_info
        xyz_i = xyz[inst_idx_i]
        sem_label_i = sem_label[inst_idx_i]
        labels, count = np.unique(sem_label_i, return_counts=True)
        instance_sem_label_i =  labels[count.argmax()]
        min_xyz_i = xyz_i.min(0)
        max_xyz_i = xyz_i.max(0)
        mean_xyz_i = xyz_i.mean(0)
        instance_info[i_, 0:3] = mean_xyz_i
        instance_info[i_, 3:6] = min_xyz_i
        instance_info[i_, 6:9] = max_xyz_i
        instance_sem_label[i_] = instance_sem_label_i
        # instance_pointnum
        instance_pointnum.append(inst_idx_i[0].size)
        instance_dict = {
            "instance_info": instance_info, 
            "instance_pointnum": instance_pointnum, 
            "instance_sem_label": instance_sem_label
        }

    return instance_num, instance_dict

CLASS_NAMES = ["wall","floor",'cabinet', 'bed', 'chair', 'sofa', 'table', 'door', 'window', 'bookshelf', 'picture', 'counter', 'desk', 'curtain', 'refrigerator', 'shower curtain', 'toilet', 'sink', 'bathtub', 'otherfurniture']
CLASS_LABELS = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 16, 24, 28, 33, 34, 36, 39])


In [35]:
# 5. convert gt annotations to batch_gt_map_cls 

gt_map_cls = []
inst_pointnum_threshold = 50

xyz = np.asarray(scannet_o3d.points)
instance_num, instance_dict = getInstanceInfo(xyz, label, instance_label)
instance_dims= instance_dict['instance_info']
instance_pointnum = instance_dict['instance_pointnum']
instance_sem_labels = instance_dict['instance_sem_label']

for inst_id in range(instance_num): 

    center = instance_dims[inst_id, 0:3]
    min = instance_dims[inst_id, 3:6]
    max = instance_dims[inst_id, 6:9]
    size = max - min 
    aabb = get_3d_box(size, 0, center)

    num_point = instance_pointnum[inst_id]
    sem_label = instance_sem_labels[inst_id]

    if sem_label >= 0 and num_point > inst_pointnum_threshold:    
        nyu40_label = CLASS_LABELS[sem_label]
        gt_map_cls.append((
            nyu40_label,
            aabb
        ))
    
batch_gt_map_cls = [gt_map_cls]

In [19]:
# read mapping from label to class name 
import csv 

mapping_nyu40id_to_nyu40class = {}
with open("./metadata/mp3d_mapping.tsv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        try:
            mapping_nyu40id_to_nyu40class[int(row['nyu40id'])] = row['nyu40class']
        except:
            print("Error:", row['nyu40id'])

Error: 


#### Calculate the metrics

In [40]:
from eval.ap_helper import APCalculator

ap_calculator = APCalculator(0.25, mapping_nyu40id_to_nyu40class)
ap_calculator.step(batch_pred_map_cls, batch_gt_map_cls)
metrics_dict = ap_calculator.compute_metrics()
for key in metrics_dict:
    print('eval %s: %f'%(key, metrics_dict[key]))

/home/junting/habitat_ws/src/struct-nav/habitat_recognition/scripts/eval/eval_det.py:159: RuntimeWarning: invalid value encountered in true_divide
  rec = tp / float(npos)


7 0.16666666666666666
3 0.45714285714285713
16 1.0
39 0.08333333333333333
4 1.0
5 nan
8 0.6666666666666666
24 1.0
34 1.0
33 1.0
14 1.0
6 1.0
9 0
12 0
eval cabinet Average Precision: 0.457143
eval bed Average Precision: 1.000000
eval chair Average Precision: nan
eval sofa Average Precision: 1.000000
eval table Average Precision: 0.166667
eval door Average Precision: 0.666667
eval window Average Precision: 0.000000
eval counter Average Precision: 0.000000
eval desk Average Precision: 1.000000
eval curtain Average Precision: 1.000000
eval refridgerator Average Precision: 1.000000
eval toilet Average Precision: 1.000000
eval sink Average Precision: 1.000000
eval otherfurniture Average Precision: 0.083333
eval mAP: nan
eval cabinet Recall: 0.571429
eval bed Recall: 1.000000
eval chair Recall: nan
eval sofa Recall: 1.000000
eval table Recall: 0.333333
eval door Recall: 0.666667
eval window Recall: 0.000000
eval counter Recall: 0.000000
eval desk Recall: 1.000000
eval curtain Recall: 1.000000

#### Visualize prediction & ground truth bounding boxes 

In [36]:
# generate predicted bounding boxes 
pred_labels = []
pred_bboxes = []
for obj in pred_map_cls:
    pred_labels.append(obj[0])
    min_bound = np.min(obj[1], 0)
    max_bound = np.max(obj[1], 0)
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
    aabb.color = (1.,0.,0)
    pred_bboxes.append(aabb)


# generate ground truth bounding boxes 
gt_labels = []
gt_bboxes = []
for obj in gt_map_cls:
    gt_labels.append(obj[0])
    min_bound = np.min(obj[1], 0)
    max_bound = np.max(obj[1], 0)
    aabb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
    aabb.color = (0.,1.,0.)
    gt_bboxes.append(aabb)



In [38]:
# visualize 
vis_list = [scannet_o3d]
# vis_list.extend(pred_bboxes)
vis_list.extend(gt_bboxes)

o3d.visualization.draw_geometries(vis_list)

### Matterport3D
1. try to reproject rtabmap back to matterport3D 
2. generate rtabmap prediction and transform it 
3. evaluate rtabmap prediction

In [41]:
mp3d_scene_dir = "/media/junting/SSD_data/habitat_data/scene_datasets/mp3d/v1/scans"
mp3d_scene_name = "17DRP5sb8fy"
scene_dir = os.path.join(mp3d_scene_dir, mp3d_scene_name)

In [42]:
############ label mapping ###################
import csv 
mapping_mpcat40id_to_nyu40id = {}

with open("./metadata/mpcat40_to_nyu40.tsv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        mapping_mpcat40id_to_nyu40id[int(row['mpcat40index'])] = int(row['nyu40id'])

mapping_nyu40id_to_nyu40class = {}
with open("./metadata/mp3d_mapping.tsv") as csvfile:
    reader = csv.DictReader(csvfile, delimiter="\t")
    for row in reader:
        try:
            mapping_nyu40id_to_nyu40class[int(row['nyu40id'])] = row['nyu40class']
        except:
            print("Error:", row['nyu40id'])

Error: 


#### 1. Test project rtabmap to matterport 3D

#### 2. generate rtabmap prediction and transform it 

In [ ]:
########## convert gt annotations to batch_gt_map_cls ############
from dataset.mp3d.utils import read_house_file, getOBB

gt_map_cls = []

house_file = os.path.join(mp3d_scene_dir, f"{mp3d_scene_name}.house")
house_dict, parse_dict = read_house_file(house_file)
# C category_index category_mapping_index category_mapping_name mpcat40_index mpcat40_name 0 0 0 0 0
mapping_rawcat_to_mpcat40 = {}
for cat_dict in house_dict['C']:
    mapping_rawcat_to_mpcat40[int(cat_dict['category_index'])] = \
        int(cat_dict['mpcat40_index'])

# O object_index region_index category_index px py pz  a0x a0y a0z  a1x a1y a1z  r0 r1 r2 0 0 0 0 0 0 0 0 
for object_dict in house_dict['O']: 

    raw_label = int(object_dict['category_index'])
    if raw_label == -1:
        nyu40_label = 0
    else:
        mpcat40_label = mapping_rawcat_to_mpcat40[raw_label]
        nyu40_label = mapping_mpcat40id_to_nyu40id[mpcat40_label]
    
    center, boxRotation, radii = getOBB(object_dict)
    o3d_obb = o3d.geometry.OrientedBoundingBox(center, boxRotation, 2*radii)
    o3d_aabb = o3d_obb.get_axis_aligned_bounding_box()
    center = o3d_aabb.get_center()
    size = o3d_aabb.get_extent()
    aabb = get_3d_box(size, 0, center)

    gt_map_cls.append((
        nyu40_label,
        aabb
    ))
batch_gt_map_cls = [gt_map_cls]